In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem

# SMILESから分子オブジェクトを生成
mol = Chem.MolFromSmiles("c1ccccc1O")  # フェノール

# 3D座標を生成
AllChem.EmbedMolecule(mol, AllChem.ETKDG())

# 構造最適化
AllChem.MMFFOptimizeMolecule(mol)

In [ ]:
from pyscf import gto, dft
# 分子オブジェクトの作成
mol = gto.M(
    atom = 'H 0 0 0; H 0 0 0.74',  # H2分子
    basis = '6-31G*',# 使用する基底関数
    unit = 'Angstrom'# 座標の単位
)

# DFT計算の実行
mf = dft.RKS(mol)
mf.xc = 'B3LYP'
mf.kernel()

# 分子軌道エネルギーの取得
mo_energy = mf.mo_energy
print(mo_energy)

In [ ]:
import random

from deap import base, creator, tools

# 適応度の定義（最大化）
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# 個体の生成
toolbox = base.Toolbox()
toolbox.register("attr_float", random.random)
toolbox.register(
    "individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=5
)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


# 評価関数
def evalOneMax(individual):
    return (sum(individual),)


toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# 初期集団の生成
population = toolbox.population(n=10)

# 進化の実行
from deap import algorithms

algorithms.eaSimple(
    population, toolbox, cxpb=0.5, mutpb=0.2, ngen=10, verbose=True
)

In [ ]:
import logging
import random
import sys
import numpy as np
from deap import algorithms, base, creator, tools
from gpu4pyscf import dft
from pyscf import gto
from rdkit import Chem, RDLogger
from rdkit.Chem import AllChem

# ログの抑制
RDLogger.DisableLog("rdApp.*")
logging.getLogger("pyscf").setLevel(logging.CRITICAL)

In [ ]:
substituents = {
    "OH": "O",        # ヒドロキシ基
    "CN": "C#N",      # シアノ基
    "NH2": "N",       # アミノ基
    "NO2": "N(=O)=O", # ニトロ基
    "OMe": "OC",      # メトキシ基
    "Me": "C",        # メチル基
    "Cl": "Cl",       # 塩素
    "F": "F",         # フッ素
    "H": "H",         # 水素
}
substituent_names = list(substituents.keys())
substituent_smiles = list(substituents.values())
num_substituents = len(substituent_names)

In [ ]:
def gen_substituted_benzene_smiles(sublist):
    """
    指定された置換基をベンゼンのパラ位に導入し、置換後のSMILESを返す。

    Parameters:
    - sublist (list of int): 6個の置換基のインデックスリスト

    Returns:
    - str: 置換基導入後のSMILES
    """
    try:
        benzene_smiles = "c1(" + substituent_smiles[sublist[0]] + ")"
        benzene_smiles += "c(" + substituent_smiles[sublist[1]] + ")"
        benzene_smiles += "c(" + substituent_smiles[sublist[2]] + ")"
        benzene_smiles += "c(" + substituent_smiles[sublist[3]] + ")"
        benzene_smiles += "c(" + substituent_smiles[sublist[4]] + ")"
        benzene_smiles += "c1" + substituent_smiles[sublist[5]]
        return benzene_smiles
    except Exception as e:
        print(sublist)
        print(f"SMILES生成中にエラーが発生しました: {e}", file=sys.stderr)
        return None

In [ ]:
def optimize_structure_rdkit(mol):
    """
    RDKitを用いて分子の3D構造を最適化します。

    Parameters:
    - mol (rdkit.Chem.rdchem.Mol): 最適化対象の分子オブジェクト

    Returns:
    - rdkit.Chem.rdchem.Mol: 最適化後の分子オブジェクト
    """
    try:
        # 3D座標の埋め込み
        params = AllChem.ETKDGv3()
        params.randomSeed = 0xF00D
        AllChem.EmbedMolecule(mol, params)

        # MMFF94力場による最適化
        AllChem.MMFFOptimizeMolecule(mol, maxIters=1000)
        return mol
    except Exception as e:
        print(
            f"RDKitによる構造最適化中にエラーが発生しました: {e}", 
            file=sys.stderr
        )
        return None

In [ ]:
def calculate_homo_lumo_gap(mol):
    """
    PySCFを用いて分子のHOMO-LUMOギャップを計算します。
    Parameters:
    - mol (rdkit.Chem.rdchem.Mol): 最適化済みの分子オブジェクト
    Returns:
    - float: HOMO-LUMOギャップ（eV）
    """
    try:
        # 3D座標を取得
        conf = mol.GetConformer()
        positions = conf.GetPositions()
        atomic_numbers = [atom.GetAtomicNum() for atom in mol.GetAtoms()]
        atoms = [
            (atomic_numbers[i], positions[i]) for i in range(len(atomic_numbers))
        ]

        # PySCFの分子オブジェクトを作成
        mol_pyscf = gto.M(atom=atoms, unit="Angstrom", basis="631G*", verbose=0)

        # DFT計算（M06汎関数）
        mf = dft.RKS(mol_pyscf)
        mf.xc = "M06"
        mf.kernel()

        # HOMO-LUMOギャップの計算
        mo_energy = mf.mo_energy
        num_electrons = mol_pyscf.nelectron
        homo_index = num_electrons // 2 - 1
        lumo_index = homo_index + 1
        if mo_energy is None:
            # 分子軌道エネルギーが取得できなかった場合、
            # ペナルティとして非常に大きなギャップを設定
            gap = 1e6
        else:
            homo_energy = mo_energy[homo_index]
            lumo_energy = mo_energy[lumo_index]
            gap = lumo_energy - homo_energy
        return gap
    except Exception as e:
        print(f"PySCFによる計算中にエラーが発生しました: {e}", file=sys.stderr)
        return None

In [ ]:
def evaluate(individual):
    """
    遺伝的アルゴリズムにおける評価関数。

    Parameters:
    - individual (list of int): 6個の置換基のインデックスリスト

    Returns:
    - tuple: HOMO-LUMOギャップの値（最小化対象）
    """

    # ペナルティ条件
    # ベンゼン環の対称性を考慮し、置換基の順序を標準化するための条件
    if individual[0] != min(individual):
        return (1e6,)
    if individual[1] > individual[5]:
        return (1e6,)

    try:
        # individualは6個の置換基インデックスのリスト
        substituted_smiles = gen_substituted_benzene_smiles(individual)
        if substituted_smiles is None:
            return (1e6,)

        # RDKitで分子オブジェクトを作成
        mol = Chem.MolFromSmiles(substituted_smiles)
        if mol is None:
            return (1e6,)
        mol = Chem.AddHs(mol)

        # 構造最適化
        mol = optimize_structure_rdkit(mol)
        if mol is None:
            return (1e6,)

        # HOMO-LUMOギャップの計算
        gap = calculate_homo_lumo_gap(mol)
        if gap is None:
            return (1e6,)

        # フィットネスとしてギャップを返す（最小化）
        print(
            f"HOMO-LUMOギャップ: {gap} eV, ",
            f"置換基: {[substituent_names[i] for i in individual]}"
        )
        return (gap,)
    except Exception as e:
        print(f"フィットネス評価中にエラーが発生しました: {e}", file=sys.stderr)
        return (1e6,)

In [ ]:
def main():
    # DEAPの設定
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # 最小化
    creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()

    # 個体は6個の置換基名（整数）のリスト
    toolbox.register("attr_sub", random.randint, 0, num_substituents - 1)
    toolbox.register(
        "individual", tools.initRepeat, creator.Individual, toolbox.attr_sub, n=6
    )
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    # 評価関数の登録
    toolbox.register("evaluate", evaluate)
    # 交叉操作の登録（個体内の置換基を交換）
    toolbox.register("mate", tools.cxTwoPoint)
    # 突然変異操作の登録（ランダムに置換基を変更）
    toolbox.register(
        "mutate", tools.mutUniformInt, low=0, up=num_substituents - 1, indpb=0.2
    )
    # 選択方法の登録（トーナメント選択）
    toolbox.register("select", tools.selTournament, tournsize=3)

    # パラメータの設定
    population_size = 20
    num_generations = 7
    crossover_prob = 0.7
    mutation_prob = 0.3

    # 初期集団の生成
    population = toolbox.population(n=population_size)

    # 遺伝的アルゴリズムの実行
    print("遺伝的アルゴリズムを開始します...")
    result = algorithms.eaSimple(
        population,
        toolbox,
        cxpb=crossover_prob,
        mutpb=mutation_prob,
        ngen=num_generations,
        stats=None,
        halloffame=None,
        verbose=True,
    )

    # 最適個体の取得
    print("最適な置換基 Top3")
    for ind, top_ind in enumerate(tools.selBest(population, k=3)):
        print(f'{ind}: {", ".join([substituent_names[i] for i in top_ind])}')

    # 最適分子の生成とギャップの計算
    best_smiles = gen_substituted_benzene_smiles(top_ind)
    if best_smiles is not None:
        mol = Chem.MolFromSmiles(best_smiles)
        mol = Chem.AddHs(mol)
        mol = optimize_structure_rdkit(mol)
        if mol is not None:
            gap = calculate_homo_lumo_gap(mol)
            if gap is not None:
                print(f"最大HOMO-LUMOギャップ: {gap:.4f} eV")
            else:
                print("HOMO-LUMOギャップの計算に失敗しました。", file=sys.stderr)
        else:
            print("構造最適化に失敗しました。", file=sys.stderr)
    else:
        print("分子生成に失敗しました。", file=sys.stderr)


if __name__ == "__main__":
    main()